In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import random

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Check if GPU is available
# Check for CUDA (NVIDIA GPU) or MPS (Apple Silicon) availability
if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [4]:
# Renommer les images capturées

for action in ['LEFT', 'RIGHT', 'FIRE', 'ENTER'] :

    dossier_cible = f"Echantillons/{action}"  # Dossier dont renommer les images

    images = os.listdir(dossier_cible)

    for i, image in enumerate(images):
        os.rename(os.path.join(dossier_cible, image), os.path.join(dossier_cible, f"{action}_{i}.jpg"))

print("Images renommées !")

Images renommées !


In [38]:
# Définir les transformations des images (redimensionnement et normalisation)
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Redimensionne les images
    transforms.ToTensor()  # Convertit en tenseur PyTorch
])

# Charger les images depuis les dossiers
dataset = datasets.ImageFolder(root="Echantillons", transform=transform, is_valid_file=lambda x: x.endswith(('.jpg')))

# Vérifier les classes détectées
print("Classes détectées :", dataset.classes)

# Créer un DataLoader pour gérer le batching
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Calculer la moyenne et l'écart type du jeu de données
mean = torch.zeros(3)
std = torch.zeros(3)
num_samples = 0

for images, _ in data_loader:
    batch_samples = images.size(0)  # Nombre d'images dans le batch
    images = images.view(batch_samples, 3, -1)  # Réorganiser pour chaque canal
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    num_samples += batch_samples

mean /= num_samples
std /= num_samples

print("Mean:", mean)
print("Std:", std)

Classes détectées : ['ENTER', 'FIRE', 'LEFT', 'RIGHT']
Mean: tensor([0.5242, 0.5160, 0.4967])
Std: tensor([0.1961, 0.1897, 0.2083])


In [30]:
## Classes :

# 0 = 'ENTER'
# 1 = 'FIRE'
# 2 = 'LEFT'
# 3 = 'RIGHT'

(tensor([[[0.6510, 0.6510, 0.6471,  ..., 0.6118, 0.6118, 0.6118],
          [0.6510, 0.6510, 0.6549,  ..., 0.6157, 0.6157, 0.6157],
          [0.6549, 0.6588, 0.6588,  ..., 0.6196, 0.6157, 0.6157],
          ...,
          [0.3451, 0.3451, 0.3451,  ..., 0.4588, 0.4549, 0.4549],
          [0.3451, 0.3490, 0.3451,  ..., 0.3765, 0.3804, 0.3882],
          [0.3451, 0.3490, 0.3451,  ..., 0.2784, 0.2784, 0.2824]],
 
         [[0.6902, 0.6902, 0.6863,  ..., 0.6392, 0.6353, 0.6353],
          [0.6902, 0.6902, 0.6941,  ..., 0.6431, 0.6392, 0.6392],
          [0.6941, 0.6980, 0.6980,  ..., 0.6431, 0.6392, 0.6392],
          ...,
          [0.3255, 0.3255, 0.3255,  ..., 0.4510, 0.4471, 0.4510],
          [0.3255, 0.3294, 0.3255,  ..., 0.3608, 0.3686, 0.3804],
          [0.3255, 0.3294, 0.3255,  ..., 0.2627, 0.2706, 0.2784]],
 
         [[0.6941, 0.6941, 0.6902,  ..., 0.6314, 0.6275, 0.6275],
          [0.6941, 0.6941, 0.6980,  ..., 0.6353, 0.6314, 0.6314],
          [0.6980, 0.7020, 0.7020,  ...,

In [32]:
len(dataset)

5635